In [1]:
from neo4j import GraphDatabase
import pandas as pd
import collections
from collections import Counter
import itertools 
import datetime
from datetime import date, timedelta
import json

from branca.colormap import linear
from ipyleaflet import Map, Heatmap, basemaps, WidgetControl, GeoJSON, Choropleth, SplitMapControl, LegendControl, Marker, Polyline, AwesomeIcon, LayersControl
import ipywidgets as widgets

# Neo4J DB Connection

In [2]:
uri = "bolt://localhost:7687"

In [3]:
driver = GraphDatabase.driver(uri, auth=("neo4j", "SFi5gS09lHiMfu9VuJTg"))

# Functions

In [4]:
def get_offenses(tx, query):
    offenses = []
    
    results = tx.run(query)
    for result in results:
        offenses.append(result["offense"])
    return offenses

In [5]:
def get_all_precincts(tx, query):
    precincts = []
    
    results = tx.run(query)
    for result in results:
        precincts.append({
            'precinct': result['precinct'],
            'complaints': result['complaints']
        })
    return precincts

In [6]:
def get_louvain(tx, query):
    locs = []
    results = tx.run(query)
        
    for result in results:
        locs.append({
            "location_id": result['location_id'],
            "latitude": result['latitude'],
            "longitude": result['longitude'],
            "community": result['community']
        })
    return locs

In [7]:
def get_page_rank(tx, query):
    locs = []
    results = tx.run(query)
        
    for result in results:
        locs.append({
            "location_id": result['location_id'],
            "latitude": result['latitude'],
            "longitude": result['longitude'],
            "score": result['score']
        })
    return locs

In [8]:
def get_nearby_locs(tx, query):
    locs = []
    results = tx.run(query)
        
    for result in results:
        locs.append({
            "l1": result['l1'],
            "l2": result['l2']
        })
    return locs

In [9]:
def get_complaints_by_loc(tx, query):
    locs = []
    results = tx.run(query)
        
    for result in results:
        locs.append({
            "location_id": result['location_id'],
            "number_of_complaints": result['number_of_complaints']
        })
    return locs

In [10]:
def get_comp_by_prec_date_offense(tx, query):
    data = []
    results = tx.run(query)
    
    for result in results:
        data.append({
            "precinct": result["precinct"],
            "complaint_date": result["complaint_date"],
            "offense": result["offense"],
            "complaints": result["complaints"],
            "complaints_per_100k_sqft": result["complaints_per_100k_sqft"]
        })
        
    return data

In [11]:
# function called by click event
def on_button_clicked(b):
    with output:
        print(list(offenses_widget.value))
        print(date_selection.value)
    
        cluster_map.clear_layers()
        cluster_map.add_layer(basemaps.CartoDB.DarkMatter)
    
    calculate_clusters(date_selection.value, list(offenses_widget.value))

In [12]:
# function called by click event
def on_compare_button_clicked(b):
    with output:    
        left_choro_map.clear_layers()
        right_choro_map.clear_layers()
        left_choro_map.add_layer(basemaps.CartoDB.DarkMatter)
        right_choro_map.add_layer(basemaps.CartoDB.DarkMatter)
    
    compare_complaint_dates(date_selection_y_m_l.value, date_selection_y_m_r.value, list(offenses_widget.value))

In [13]:
def calculate_clusters(dates, offenses):
    
    from_date = f"'{dates[0]}'"
    to_date = f"'{dates[1]}'"
    
    louvain_query = 'CALL gds.louvain.stream({  \n\
    nodeQuery: "MATCH (l:Location)<-[:LOCATED_AT]-(c:Complaint)-[:COMMITTED_OFFENSE]->(o:Offense) ' \
    'where o.offense in '+ str(offenses) + ' and c.complaint_date >= ' + from_date + ' and c.complaint_date <= ' + to_date + '  return distinct id(l) as id",\n\
    relationshipQuery: "MATCH (c:Complaint)-[:LOCATED_AT]->(l1:Location)-[:LOCATED_NEARBY]->(l2:Location) return id(l1) as source, id(l2) as target",\n\
    validateRelationships: false,\n\
    maxIterations: 50})\n\
    yield nodeId, communityId\n\
    RETURN gds.util.asNode(nodeId).location_id AS location_id, gds.util.asNode(nodeId).latitude AS latitude, gds.util.asNode(nodeId).longitude AS longitude, communityId as community'
    
    number_of_complaints_query = 'MATCH (l:Location)<-[:LOCATED_AT]-(c:Complaint)-[:COMMITTED_OFFENSE]->(o:Offense) where o.offense in '+ str(offenses) + ' and c.complaint_date >= ' + from_date + ' and c.complaint_date <= ' + to_date + ' return distinct l.location_id as location_id, count(c) as number_of_complaints'
    
    
    with driver.session() as session:
        louvain = session.read_transaction(get_louvain, louvain_query)
        louvain_df = pd.DataFrame(louvain)
        
        louvain_df['latitude'] = louvain_df.latitude.astype(float)
        louvain_df['longitude'] = louvain_df.longitude.astype(float)
        
        complaints = session.read_transaction(get_complaints_by_loc, number_of_complaints_query)
        complaints_df = pd.DataFrame(complaints)
        
        df_merge = louvain_df.merge(complaints_df, on=['location_id'])
        
        clusters = df_merge.groupby('community').agg({'latitude': lambda x: x.mean(), 'longitude': lambda x: x.mean(),'number_of_complaints': lambda x: x.sum()})
        
        lat_lon = list(clusters.apply(lambda x: (x['latitude'], x['longitude'],x['number_of_complaints']), axis=1))
        print(lat_lon)
                
        cluster_map.add_layer(Heatmap(locations=lat_lon, radius=20))

In [14]:
def compare_complaint_dates(left_date, right_date, offenses):
    
    left_df = comp_prec_df.loc[(comp_prec_df.complaint_date==left_date) & (comp_prec_df.offense.isin(offenses)),['precinct','complaints']]
    right_df = comp_prec_df.loc[(comp_prec_df.complaint_date==right_date) & (comp_prec_df.offense.isin(offenses)),['precinct','complaints']]
    
    left_dict = dict(zip(left_df['precinct'].tolist(), left_df['complaints'].tolist()))
    right_dict = dict(zip(right_df['precinct'].tolist(), right_df['complaints'].tolist()))
    
    
    left_dict = dict((x, left_dict.get(x, 0) + all_precincts_dict.get(x, 0)) for x in set(left_dict)|set(all_precincts_dict))
    right_dict = dict((x, right_dict.get(x, 0) + all_precincts_dict.get(x, 0)) for x in set(right_dict)|set(all_precincts_dict))
    
    left_layer = Choropleth(
        geo_data=precincts_json,
        choro_data=left_dict,
        key_on='id',
        colormap=linear.YlOrRd_04,
        border_color='black',
        style={'fillOpacity': 0.8})
    
    right_layer = Choropleth(
        geo_data=precincts_json,
        choro_data=right_dict,
        key_on='id',
        colormap=linear.YlOrRd_04,
        border_color='black',
        style={'fillOpacity': 0.8})
    
    
    left_choro_map.add_layer(left_layer)
    right_choro_map.add_layer(right_layer)


In [15]:
# function called by click event
def on_page_rank_button_clicked(b):
    with output:    
        page_rank_map.clear_layers()
        page_rank_map.add_layer(basemaps.CartoDB.DarkMatter)
    
    page_rank_locations(int_slider.value, list(offenses_widget.value))

In [16]:
def add_marker(x):
    coord = (x['latitude'], x['longitude'])
    
    icon1 = AwesomeIcon(
        name='bullseye',
        marker_color='red',
        icon_color='black',
        spin=False)
    
    marker = Marker(icon=icon1,location=coord, title='Location: ' + str(coord)+'\n'+'Page Rank: ' +str(x['score']), draggable=False)
    page_rank_map.add_layer(marker)
    

In [17]:
def add_nearby_marker(x):
    coord = tuple(x['l2'])
    
    icon1 = AwesomeIcon(
        name='map-pin',
        marker_color='lightblue',
        icon_color='black',
        spin=False)
    
    marker = Marker(icon=icon1, location=coord, draggable=False)
    
    line = Polyline(locations=[x['l1'],x['l2']],
    color="white",
    fill=False)
    
    page_rank_map.add_layer(line)
    page_rank_map.add_layer(marker)
    

In [18]:
def page_rank_locations(days_back, offenses):
    
    latest = datetime.datetime(2021, 3, 31)
    d = datetime.timedelta(days = days_back)
    a = latest - d
    print(a)
    from_date = a.strftime('%Y-%m-%d')  
    
    from_date = f"'{from_date}'"
    
    page_rank_query = 'CALL gds.pageRank.stream({ \n\
    nodeQuery: "match (l:Location) return distinct id(l) as id",\n \
    relationshipQuery: "match (o:Offense)<-[:COMMITTED_OFFENSE]-(c:Complaint)-[:LOCATED_AT]->(l:Location)-[:LOCATED_NEARBY]->(l2:Location) where o.offense in ' + str(offenses) + ' and c.complaint_date >= ' + from_date + ' return id(l) as source, id(l2) as target",\n\
    validateRelationships: false})\n\
    YIELD nodeId, score\n\
    with gds.util.asNode(nodeId).location_id AS location_id, gds.util.asNode(nodeId).latitude AS latitude, gds.util.asNode(nodeId).longitude AS longitude, score\n\
    where location_id is not null\n\
    return location_id, latitude, longitude, score\n\
    order by score desc\n\
    limit 10'
    
    print(page_rank_query)
    
    with driver.session() as session:
        page_rank = session.read_transaction(get_page_rank, page_rank_query)
        page_rank_df = pd.DataFrame(page_rank)
        
        page_rank_df['latitude'] = page_rank_df.latitude.astype(float)
        page_rank_df['longitude'] = page_rank_df.longitude.astype(float)
        
        
        
        nearby_locs_query = 'match (l1:Location)-[ln:LOCATED_NEARBY]-(l2:Location)<-[:LOCATED_AT]-(c:Complaint)-[:COMMITTED_OFFENSE]->(o:Offense) \n\
        where l1.location_id in '+ str(list(page_rank_df.location_id.unique())) + 'and c.complaint_date >= ' + from_date + ' and o.offense in '+ str(offenses) + ' with \
        apoc.convert.toFloat(l1.latitude) as l1_lat,apoc.convert.toFloat(l1.longitude) as l1_lon, apoc.convert.toFloat(l2.latitude) as l2_lat,apoc.convert.toFloat(l2.longitude) as l2_lon \
        return distinct [l1_lat,l1_lon] as l1,[ l2_lat,l2_lon] as l2'
        
        nearby_locs = session.read_transaction(get_nearby_locs, nearby_locs_query)
        nearby_locs_df = pd.DataFrame(nearby_locs)
        
        nearby_locs_df.apply(add_nearby_marker, axis=1)
        
        page_rank_df.apply(add_marker, axis=1)

# Properties

## Widget Properties

In [19]:
# output widget
output = widgets.Output()

## Map Properties

In [20]:
center = [40.7128, -74.0060]
zoom = 11

## Date Properties

sdate = date(2019,1,1)   # start date
edate = date(2021,3,31)   # end date

date_range = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
date_range = [d.strftime('%Y-%m-%d') for d in date_range]

date_range2 = pd.date_range(sdate,edate-timedelta(days=1),freq='m')
date_range2 = [d.strftime('%Y-%m') for d in date_range2]
print(date_range2)

In [21]:
# date properties
sdate = date(2019,1,1)   # start date
edate = date(2021,3,31)   # end date

# first map
date_range = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
date_range = [d.strftime('%Y-%m-%d') for d in date_range]

# second map
date_range2 = pd.date_range(sdate,edate-timedelta(days=1),freq='m')
date_range2 = [d.strftime('%Y-%m') for d in date_range2]

# All Precincts Query

In [22]:
# needed so every entry in the choropleth map has a value, 0 by default
all_precincts_query = 'match (p:PrecinctName) return distinct p.precinct as precinct, 0 as complaints'

In [23]:
with driver.session() as session:
    all_precincts = session.read_transaction(get_all_precincts, all_precincts_query)

In [24]:
all_precincts_df = pd.DataFrame(all_precincts)
all_precincts_df.head()

,precinct,complaints
0,Precinct 75,0
1,Precinct 110,0
2,Precinct 52,0
3,Precinct 43,0
4,Precinct 46,0


In [25]:
all_precincts_dict = dict(zip(all_precincts_df['precinct'].tolist(), all_precincts_df['complaints'].tolist()))

# Load Precincts GeoJSON

In [26]:
with open('./data/gis/geojson/Police Precincts.geojson') as file:
    precincts_json = json.load(file)

In [27]:
for x in precincts_json['features']:
    x['precinct'] = 'Precinct ' + x['properties']['precinct']
    x['id'] = 'Precinct ' + x['properties']['precinct']
    

# Complaints by Precinct, Date, and Offense

In [28]:
# comp_by_prec_date_offense_query = 'match (o:GeneralOffense)<-[:COMMITTED_OFFENSE]-(c:Complaint)<-[:REPORTED]-(p:PrecinctName) \
# return p.precinct as precinct, apoc.temporal.format(date(c.complaint_date),"yyyy-MM") as complaint_date, o.offense as offense, count(c) as complaints \
# order by precinct'

In [29]:
comp_by_prec_date_offense_query = 'match (o:GeneralOffense)<-[:COMMITTED_OFFENSE]-(c:Complaint)<-[:REPORTED]-(p:PrecinctName) \
with p.precinct as precinct, apoc.temporal.format(date(c.complaint_date),"yyyy-MM") as complaint_date, o.offense as offense, apoc.convert.toFloat(p.area) as area, count(c) as complaints \
return precinct, complaint_date, offense, complaints, complaints/area  * 100000 as complaints_per_100k_sqft order by precinct'

In [30]:
with driver.session() as session:
    comp_prec_date_off = session.read_transaction(get_comp_by_prec_date_offense, comp_by_prec_date_offense_query)

In [31]:
comp_prec_df = pd.DataFrame(comp_prec_date_off)

In [32]:
comp_prec_df.head()

,precinct,complaint_date,offense,complaints,complaints_per_100k_sqft
0,Precinct 1,2019-04,GRAND LARCENY,66,0.139488
1,Precinct 1,2020-08,GRAND LARCENY,61,0.128921
2,Precinct 1,2020-05,CRIMINAL MISCHIEF & RELATED OF,21,0.044383
3,Precinct 1,2020-01,GRAND LARCENY,94,0.198665
4,Precinct 1,2020-08,BURGLARY,14,0.029588


# Offenses

In [33]:
offenses_query = 'match (o:Offense)<-[:COMMITTED_OFFENSE]-(c:Complaint) \
where c.complaint_date >= ' + "'2019-01-01'" + ' and c.complaint_date <= ' + "'2021-03-31'" + 'return distinct o.offense as offense'

In [34]:
with driver.session() as session:
    offenses = session.read_transaction(get_offenses, offenses_query)

In [35]:
offenses = sorted(offenses)


# Testing

In [36]:
test_map = Map(basemap=basemaps.CartoDB.DarkMatter, center=center, zoom=zoom, layout=widgets.Layout(width='100%', height='800px'))

test_df = comp_prec_df.loc[(comp_prec_df.complaint_date== '2019-02') & (comp_prec_df.offense.isin(['GRAND LARCENY'])),['precinct','complaints']]
test_dict = dict(zip(test_df['precinct'].tolist(), test_df['complaints'].tolist()))
test_dict = dict((x, test_dict.get(x, 0) + all_precincts_dict.get(x, 0)) for x in set(test_dict)|set(all_precincts_dict))
test_layer = Choropleth(
        geo_data=precincts_json,
        choro_data=test_dict,
        key_on='id',
        colormap=linear.YlOrRd_04,
        value_min=0,
        value_max=100,
        border_color='black',
        style={'fillOpacity': 0.8})


test2_df = comp_prec_df.loc[(comp_prec_df.complaint_date== '2021-02') & (comp_prec_df.offense.isin(['GRAND LARCENY'])),['precinct','complaints']]
test2_dict = dict(zip(test2_df['precinct'].tolist(), test2_df['complaints'].tolist()))
test2_dict = dict((x, test2_dict.get(x, 0) + all_precincts_dict.get(x, 0)) for x in set(test2_dict)|set(all_precincts_dict))
test2_layer = Choropleth(
        geo_data=precincts_json,
        choro_data=test2_dict,
        key_on='id',
        colormap=linear.YlOrRd_04,
        value_min=0,
        value_max=100,
        border_color='black',
        style={'fillOpacity': 0.8})

test_map.add_layer(test_layer)

# Cluster Map

In [37]:
cluster_map = Map(basemap=basemaps.CartoDB.DarkMatter, center=center, zoom=zoom, layout=widgets.Layout(width='100%', height='800px'))
heatmap = Heatmap(locations=[], radius=20)

# widget to select multiple offenses for the analysis
offenses_widget = widgets.SelectMultiple(
    options=offenses,
    value=[offenses[0]],
    description='Offenses',
    disabled=False,
    layout={'width': '500px'}
)

# widget to generate the clusters in heat map based on the offenses and dates selected
button = widgets.Button(
    description='Show Clusters',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show clusters',
    icon='network-wired' # (FontAwesome names without the `fa-` prefix)
)

# date select widget to select a range of dates for the analysis
date_selection = widgets.SelectionRangeSlider(
    options=date_range,
    index=(0, len(date_range)-1),
    description='Date Range',
    disabled=False,
    layout={'width': '700px'}
)

slider = widgets.IntSlider()



offenses_control = WidgetControl(widget=offenses_widget, position='bottomleft', display="flex", width=500)
date_control = WidgetControl(widget=date_selection, position='bottomright', display="flex", width=700, min_width=500, max_width=1000)
button_control = WidgetControl(widget=button, position='topright')
# slider_control = WidgetControl(widget=hbox, position='bottomright')  

cluster_map.add_control(offenses_control)
cluster_map.add_control(date_control)
cluster_map.add_control(button_control)
# m.add_control(slider_control)

# button click event
button.on_click(on_button_clicked)

In [55]:
cluster_map

Map(bottom=197522.0, center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

# Choropleth Map of Complaints

In [39]:
left_choro_map = Map(basemap=basemaps.CartoDB.DarkMatter, center=center, zoom=zoom, layout=widgets.Layout(width='100%', height='800px'))
right_choro_map = Map(basemap=basemaps.CartoDB.DarkMatter, center=center, zoom=zoom, layout=widgets.Layout(width='100%', height='800px'))

In [40]:
legend = LegendControl({"Low":"#e0d987", "Medium":"#d9822b", "High":"#c23030"}, name="Legend", position="bottomright")
right_choro_map.add_control(legend)

In [41]:
left_choro_map.add_control(offenses_control)

In [42]:
compare_button = widgets.Button(
    description='Compare Dates',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Compare Complaint Dates',
    icon='calendar-alt' # (FontAwesome names without the `fa-` prefix)
)

compare_button_control = WidgetControl(widget=compare_button, position='topright')
compare_button.on_click(on_compare_button_clicked)

In [43]:
right_choro_map.add_control(compare_button_control)

In [44]:
# date select widget to select a range of dates for the analysis
date_selection_y_m_l = widgets.SelectionSlider(
    options=date_range2,
    index=len(date_range2) -2,
    description='Date Range',
    disabled=False,
    layout={'width': '400px'}
)

date_selection_y_m_r = widgets.SelectionSlider(
    options=date_range2,
    index=len(date_range2)-1,
    description='Date Range',
    disabled=False,
    layout={'width': '400px'}
)

date_y_m_left_control = WidgetControl(widget=date_selection_y_m_l, position='topleft', display="flex", width=400, min_width=400)
date_y_m_right_control = WidgetControl(widget=date_selection_y_m_r, position='topleft', display="flex", width=400, min_width=400)

left_choro_map.add_control(date_y_m_left_control)
right_choro_map.add_control(date_y_m_right_control)

In [45]:
widgets.HBox([left_choro_map, right_choro_map])

# Central Locations of Offenses

In [46]:
page_rank_map = Map(basemap=basemaps.CartoDB.DarkMatter, center=center, zoom=zoom, layout=widgets.Layout(width='100%', height='800px'))

In [47]:
page_rank_map.add_control(offenses_control)

In [48]:
int_slider = widgets.IntSlider(
    value=30,
    min=0,
    max=90,
    step=1,
    description='Days Back:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

In [49]:
int_slider_control = WidgetControl(widget=int_slider, position='topright', display="flex", width=700)

In [50]:
page_rank_map.add_control(int_slider_control)

In [51]:
page_rank_button = widgets.Button(
    description='Get Locations',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Get Central Locations',
    icon='fa-location-arrow'
)


In [52]:
page_rank_button_control = WidgetControl(widget=page_rank_button, position='topright')
page_rank_button.on_click(on_page_rank_button_clicked)

In [53]:
page_rank_map.add_control(page_rank_button_control)

In [54]:
page_rank_map

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…